# Simplex Method

In [2]:
import numpy as np

## Exercises 1-6

In [262]:
class LinOptProb:
    '''
    A Linear Optimization Problem Class. 
    
    Attributes:
    A = matrix of coefficients in constraint
    b = Vector of constraint vals
    c = Vector for objective function
    L = vector of basic variables
    M, N = shape of matrix A - define dimension
    '''
    def __init__(self, A, b, c):
        self.A = A
        self.b = b
        self.c = c
        #Define dimension
        self.M = len(b)
        self.N = len(c)
        self.L = np.append(np.linspace(self.N, self.N+self.M-1, self.M), \
                           np.linspace(0, self.N-1, self.N))
        self.L = np.array(self.L, dtype=int)
        if np.min(b) < 0: 
            raise ValueError("Problem is not feasible at the origin. \
            Go solve an auxillary problem and come back later.")
    
    def createtab(self):
        '''
        This method creates the initial tableau.
        '''
        Abar = np.hstack([self.A, np.eye(self.M)])
        cbar = np.hstack([self.c, np.zeros(self.M)]) 
        top = np.hstack([np.array([0]), -cbar.T, np.array([1])])
        bot = np.column_stack([self.b, Abar, np.zeros(self.M)])
        init_tableau = np.vstack([top, bot])
        self.tableau = init_tableau
        
    def plocator(self):
        '''
        This method locates the pivot column/row for the tableau
        '''
        entindex = np.argwhere(self.tableau[0,1:] < 0)[0]
        print(entindex)
        #Create mask for positive values in entindex
        pivcol = self.tableau[:, entindex+1].T
        conscol = self.tableau[:, 0] 
        mask = pivcol <= 0
        #if np.all(self.tableau[:, entindex+1]) <= 0:
        #    print(self.tableau[:, entindex+1], np.all(self.tableau[:, entindex+1]) <= 0)
        #    raise ValueError("Unbounded!")    
        pivcol[mask] = np.nan
        leavindex = np.array([np.nanargmin(conscol/pivcol)])[0]
        #The index above implements Bland's Rule!
        return entindex, leavindex
    
    def pivot(self):
        '''
        Performs one pivot at the entindex and leavindex specified by
        plocator. Note that the pivot takes place in the column 
        entindex+1 and the row leavindex.
        '''
        print("Here comes a pivot. \n")
        print("\n BEFORE: \n ", self.tableau)
        entindex, leavindex = self.plocator()
        #Swap basic tracker around (weird indexing, I know...)
        self.L[self.M + entindex], self.L[leavindex-1] = self.L[leavindex-1], self.L[self.M + entindex]
        pval = self.tableau[leavindex, entindex+1]
        #divide pivot row by pivotval
        self.tableau[leavindex, :] = self.tableau[leavindex, :]/pval
        #Zero out other columns.
        for i in range(self.M+1):
            if i == leavindex:
                #Do nothing to the pivot-row
                i == i
            else:
                #determine what factor for pivot row is necessary to zero.
                factor = self.tableau[i,entindex+1]
                self.tableau[i, :] = self.tableau[i, :] - factor * self.tableau[leavindex, :]
        print("\n AFTER: \n ", self.tableau)
                
    def solve(self):
        '''
        Function pivots the tableau until it cannot pivot any more, and then
        reads out the optimal values as a dictionary.
        
        NOTE: This only gives the right answer when you run it the *first* time...
        Or on other "odd numbered" times.
        '''
        self.createtab()
        #Define tracker which will switch to true when pivoting is done
        check = (np.all(self.tableau[0,:] >= 0)) 
        while check==False:
            self.pivot()
            check = np.all(self.tableau[0,:] >= 0)
        #Read out correct vals
        maximizer = self.tableau[0,0]
        basicdict = {}
        for i in range(self.M):
            basicdict[Prob.L[i]] = \
            np.round_(self.tableau[:,0].T @ self.tableau[:,Prob.L[i]+1], 3)
        nonbasicdict = {}
        for j in range(self.N):
            nonbasicdict[Prob.L[self.M+j]] = 0
        output = (maximizer, basicdict, nonbasicdict)
        return output

In [263]:
# Define testing matrices
Atest = np.array([[1, -1], [3, 1], [4, 3]])
btest = np.array([2, 5, 7])
ctest = np.array([3, 2])

#Initialize problem class
Prob = LinOptProb(Atest, btest, ctest)

In [264]:
result = Prob.solve()
print(result)
Prob.tableau
#Prob.L
#Prob.L

Here comes a pivot. 


 BEFORE: 
  [[ 0. -3. -2. -0. -0. -0.  1.]
 [ 2.  1. -1.  1.  0.  0.  0.]
 [ 5.  3.  1.  0.  1.  0.  0.]
 [ 7.  4.  3.  0.  0.  1.  0.]]
[0]

 AFTER: 
  [[ 5.          0.         -1.          0.          1.          0.
   1.        ]
 [ 0.33333333  0.         -1.33333333  1.         -0.33333333  0.
   0.        ]
 [ 1.66666667  1.          0.33333333  0.          0.33333333  0.
   0.        ]
 [ 0.33333333  0.          1.66666667  0.         -1.33333333  1.
   0.        ]]
Here comes a pivot. 


 BEFORE: 
  [[ 5.          0.         -1.          0.          1.          0.
   1.        ]
 [ 0.33333333  0.         -1.33333333  1.         -0.33333333  0.
   0.        ]
 [ 1.66666667  1.          0.33333333  0.          0.33333333  0.
   0.        ]
 [ 0.33333333  0.          1.66666667  0.         -1.33333333  1.
   0.        ]]
[1]

 AFTER: 
  [[ 5.2  0.   0.   0.   0.2  0.6  1. ]
 [ 0.6  0.   0.   1.  -1.4  0.8  0. ]
 [ 1.6  1.   0.   0.   0.6 -0.2  0. ]
 [ 0.2  0

array([[ 5.2,  0. ,  0. ,  0. ,  0.2,  0.6,  1. ],
       [ 0.6,  0. ,  0. ,  1. , -1.4,  0.8,  0. ],
       [ 1.6,  1. ,  0. ,  0. ,  0.6, -0.2,  0. ],
       [ 0.2,  0. ,  1. ,  0. , -0.8,  0.6,  0. ]])

## Exercise 7

In [265]:
#Read in Data
data = np.load('Data/productMix.npz')
A = data['A']
p = data['p']
m = data['m']
d = data['d']

A = np.vstack((A, np.identity(len(p))))
b = np.hstack((m, d))
c = p

In [266]:
LinOptProb(A, b, c).solve()

Here comes a pivot. 


 BEFORE: 
  [[ 0.00e+00 -2.50e+02 -2.15e+02 -2.75e+02 -1.80e+02 -0.00e+00 -0.00e+00
  -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00  1.00e+00]
 [ 4.00e+00  1.20e-01  1.80e-01  1.30e-01  7.00e-02  1.00e+00  0.00e+00
   0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 4.00e+00  1.50e-01  1.20e-01  1.30e-01  1.10e-01  0.00e+00  1.00e+00
   0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 4.00e+00  1.00e-01  1.00e-01  1.00e-01  1.20e-01  0.00e+00  0.00e+00
   1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 1.00e+01  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 2.00e+01  0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 1.20e+01  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00]
 [ 1.00e+01  0.00e+0

IndexError: index 5 is out of bounds for axis 0 with size 5

In [261]:
x = np.array([[-2.5e+02],[ 1.2e-01],[ 1.5e-01],[ 1.0e-01],[ 1.0e+00],[ 0.0e+00],[ 0.0e+00],[ 0.0e+00]])

np.all(x) == 0

array([[-2.5e+02],
       [ 1.2e-01],
       [ 1.5e-01],
       [ 1.0e-01],
       [ 1.0e+00],
       [ 0.0e+00],
       [ 0.0e+00],
       [ 0.0e+00]])